In [20]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [21]:
# data
data = pd.read_csv("data/songs_with_genre.csv",index_col=0,header=[0,1,2])
data.head()

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
3             1.888963  0.760539  0.345297  2.295201  1.654031  0.067592   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
10            3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
134           0.918445  0.674147  0.577818  1.281117  0.933746  0.078177   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        05        06   
track_id                                            ...                       
2           1.481593  2.691455  0.866868  1.341231  ...  0.012226  0.012111   
3           1.366848  1.054094  0.108103  0.619185  ...  0.014212  0.017740   
5          -0.923192 -0.927232  0.666617  1.038546  ...  0.012691  0.014759   
10          1.770694  1.604566  0.521217  1.982386  ...  0.017952  0.013921   
134         1.199204 -0.175223  0.925482  1.438509  ...  0.016322  0.015819   

feature           zcr                                                         \
statistics   kurtosis       max      mean    median  min      skew       std   
number             01        01        01        01   01        01        01   
track_id                                                                       
2            5.758890  0.459473  0.085629  0.071289  0.0  2.089872  0.061448   
3            2.824694  0.466309  0.084578  0.063965  0.0  1.716724  0.069330   
5            6.808415  0.375000  0.053114  0.041504  0.0  2.193303  0.044861   
10          21.434212  0.452148  0.077515  0.071777  0.0  3.542325  0.040800   
134          4.731087  0.419434  0.064370  0.050781  0.0  1.806106  0.054623   

feature                genre_id  
statistics Unnamed: 519_level_1  
number     Unnamed: 519_level_2  
track_id                         
2                            20  
3                            20  
5                            20  
10                            9  
134                          20  

[5 rows x 519 columns]

In [22]:
data.columns

MultiIndex([('chroma_cens',             'kurtosis',                   '01'),
            ('chroma_cens',             'kurtosis',                   '02'),
            ('chroma_cens',             'kurtosis',                   '03'),
            ('chroma_cens',             'kurtosis',                   '04'),
            ('chroma_cens',             'kurtosis',                   '05'),
            ('chroma_cens',             'kurtosis',                   '06'),
            ('chroma_cens',             'kurtosis',                   '07'),
            ('chroma_cens',             'kurtosis',                   '08'),
            ('chroma_cens',             'kurtosis',                   '09'),
            ('chroma_cens',             'kurtosis',                   '10'),
            ...
            (    'tonnetz',                  'std',                   '05'),
            (    'tonnetz',                  'std',                   '06'),
            (        'zcr',             'kurtosis',         

In [23]:
data.iloc[:,-1:]

feature,genre_id
statistics,Unnamed: 519_level_1
number,Unnamed: 519_level_2
track_id,
2,20
3,20
5,20
10,9
134,20
...,...
155315,11


In [24]:
# TODO check the data upload
# targets
generes = data["genre_id"]
# generes['genre_id']
data.drop(columns=["genre_id"], inplace=True)


C:\Users\talia\AppData\Local\Temp\ipykernel_10916\2201985262.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  data.drop(columns=["genre_id"], inplace=True)


In [25]:
generes.columns = range(generes.shape[1])
generes

,0
track_id,
2,20
3,20
5,20
10,9
134,20
...,...
155315,11
155316,11
155317,11


In [26]:
generes.iloc[3]

0    9
Name: 10, dtype: int64

In [27]:
inx = pd.read_csv("data/genres_index.csv", index_col=0)
inx.head()

,genre_index
Avant-Garde,0
International,1
Blues,2
Jazz,3
Classical,4


In [37]:
vocabulary_size = 518
categories = 161 # 161
x = tf.placeholder(tf.float32, [None, vocabulary_size])
y_ = tf.placeholder(tf.float32, [None, categories])
W = tf.Variable(tf.zeros([vocabulary_size, categories]))
b = tf.Variable(tf.zeros([categories]))

z = tf.matmul(x, W) + b
pred = tf.nn.softmax(z)
# loss = -tf.reduce_mean(y_ * tf.log(pred))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(y_, z))

update = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)

In [29]:
data_x = data
targets = OneHotEncoder().fit_transform(inx).toarray()
print(generes)
# print(data_x)

           0
track_id    
2         20
3         20
5         20
10         9
134       20
...       ..
155315    11
155316    11
155317    11
155318    11
155319    11

[49598 rows x 1 columns]


In [31]:
# generes[0] = generes[0]-1
data_y = [targets[i-1] for i in generes[0]]
max(generes[0])

161

In [32]:
len(data_y)

49598

In [38]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [39]:
for i in range(0, 1001):
    sess.run(update, feed_dict={x:data_x, y_:data_y})
    if i % 100 == 0 :
        print('Iteration:' , i , '\n',
              ' W:' , sess.run(W) , '\n',
              ' b:' , sess.run(b), '\n',
              'loss:', loss.eval(session=sess, feed_dict = {x:data_x, y_:data_y}))


Iteration: 0 
  W: [[-1.2397388e-07 -2.2343107e-07 -1.4449859e-06 ... -1.5255611e-07
  -1.5255611e-07  5.5968252e-07]
 [-1.0758078e-07 -2.7230178e-07 -1.4994737e-06 ... -1.0587520e-07
  -1.0587520e-07 -6.5371728e-07]
 [-1.5304096e-07 -1.9454353e-07 -1.5894444e-06 ... -1.7860205e-07
  -1.7860205e-07  6.1611576e-07]
 ...
 [-1.1823883e-09  2.2836784e-10  1.8891828e-09 ... -1.6539530e-09
  -1.6539530e-09  2.1863258e-08]
 [-1.1033284e-06  2.1740009e-06  5.9632880e-06 ... -1.8797855e-06
  -1.8797855e-06  2.3943192e-05]
 [-1.6898998e-08  1.5729123e-08  3.1355018e-08 ... -2.4990245e-08
  -2.4990245e-08  2.1655256e-07]] 
  b: [-3.9929668e-07  5.3004061e-07  1.8587235e-06 -6.2114196e-07
 -6.2114196e-07  4.9575715e-07 -2.2995444e-07  4.0810469e-06
 -6.2114196e-07  2.7970680e-05 -5.7273979e-07 -2.6827138e-07
  1.8273991e-05 -6.2114196e-07  5.0308995e-06 -6.2114196e-07
 -6.2114196e-07  2.3179915e-07  6.5411955e-06 -6.2114196e-07
 -6.2114196e-07 -6.2114196e-07 -6.2114196e-07 -6.2114196e-07
 -6.21141